In [2]:
import pandas as pd
from typing import Dict

In [3]:
df = pd.read_excel('Nex3_Fuse_for_gui.xlsx')

In [4]:
df.head()

,Bits,Parameter,Value,Defaults,Description,Value_hex,start_bit,ADD
0,3,inventory_period,5,5,Controls the rate of inventory packets,0x5,0,100
1,2,tamper_period,1,1,Controls the rate of tamper packets\n,0x1,3,100
2,3,eas_period,3,3,Controls the rate of EAS search pattern\n,0x3,5,100
3,2,xtal_stabil_time,0,0,Configure time to count for Xtal stabilization...,0x0,8,100
4,1,bg_replenish_en,0,0,Bandgap charge pulse prior to any TX. This sho...,0x0,10,100


In [5]:
def get_command(changes: Dict[str, int]) -> str:
    df = (
        pd.read_excel('Nex3_Fuse_for_gui.xlsx')
        .assign(new_val=lambda d: d.Value)
    )
    for parameter, new_val in changes.items():
        entry = df.loc[lambda d: d.Parameter == parameter]
        assert len(entry) == 1, f'{len(entry)}'
        assert 0 <= new_val < 2 ** entry.iloc[0]['Bits'], f'{entry.iloc[0]["Bits"]}'
        df.loc[lambda d: d.Parameter == parameter, 'new_val'] = new_val
    df = (
        df
        .assign(send_value = lambda d: d.new_val ^ d.Value)
        .assign(f = lambda d: d.Bits.map(lambda b: f'0{b}b'))
        .sort_values('start_bit')
    )
    elements = [
        f'{send_value:>{f}}'
        for send_value, f in zip(df['send_value'], df['f'])
    ]

    as_binary = ''.join(elements) 
    return f'0x{int(as_binary, 2):0x}'

In [6]:
get_command({'eas_period': 6})

'0xa0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'